In [2]:
import os
import cv2
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import mediapipe as mp

In [3]:
# Frame extraction function
def extract_frames(video_path, output_folder, frame_rate=1):
    video_capture = cv2.VideoCapture(video_path)
    frame_count = 0
    success = True

    while success:
        success, frame = video_capture.read()
        if success and frame_count % frame_rate == 0:
            frame_filename = os.path.join(output_folder, f"frame_{frame_count}.jpg")
            cv2.imwrite(frame_filename, frame)
        frame_count += 1

    video_capture.release()

In [4]:
import cv2
import mediapipe as mp
import numpy as np

mp_pose = mp.solutions.pose

def extract_skeleton_keypoints(image_path):
    image = cv2.imread(image_path)
    with mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5) as pose:
        results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        if results.pose_landmarks:
            keypoints = []
            for landmark in results.pose_landmarks.landmark:
                keypoints.append((landmark.x, landmark.y))
            return image_path, np.array(keypoints).flatten()
        else:
            return image_path, None

In [5]:
import os
import pandas as pd

def save_keypoints_to_csv(images_folder, csv_filename):
    data = []
    for image_filename in os.listdir(images_folder):
        image_path = os.path.join(images_folder, image_filename)
        image_path, keypoints = extract_skeleton_keypoints(image_path)
        if keypoints is not None:
            data.append([image_path] + keypoints.tolist())
    
    # Create a DataFrame and save to CSV
    columns = ['image_path'] + [f'keypoint_{i}' for i in range(len(data[0]) - 1)]
    df = pd.DataFrame(data, columns=columns)
    df.to_csv(csv_filename, index=False)

# Example usage
save_keypoints_to_csv('keypoints', 'keypoints_lstm.csv')

IndexError: list index out of range

In [6]:
# Dataset preparation function
def prepare_dataset(frames_folder, label, sequence_length=10):
    data = []
    sequence = []
    for frame_filename in sorted(os.listdir(frames_folder)):
        frame_path = os.path.join(frames_folder, frame_filename)
        keypoints = extract_skeleton_keypoints(frame_path)
        if keypoints is not None:
            sequence.append(keypoints)
            if len(sequence) == sequence_length:
                data.append((np.array(sequence), label))
                sequence = []
    return data


In [7]:
# Skeleton keypoints extraction function
mp_pose = mp.solutions.pose

def extract_skeleton_keypoints(image_path):
    image = cv2.imread(image_path)
    with mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.5) as pose:
        results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        if results.pose_landmarks:
            keypoints = []
            for landmark in results.pose_landmarks.landmark:
                keypoints.append((landmark.x, landmark.y))
            return np.array(keypoints).flatten()
        else:
            return None


In [8]:
# LSTM-based pose classifier model
class PoseLSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers=2):
        super(PoseLSTMClassifier, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x):
        h0 = torch.zeros(2, x.size(0), hidden_dim).to(x.device)
        c0 = torch.zeros(2, x.size(0), hidden_dim).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return torch.sigmoid(out)


In [13]:
## FOR MULTIPLE CLASSES
from torch.utils.data import DataLoader, TensorDataset, random_split

In [17]:
if __name__ == "__main__":
    # Path to the folder containing the videos
    #folder = 'D:/anantasana'
    #class_folders = ['Anantasana Right Steps', 'Ardhakati Chakrasana Right Steps']  # Update with actual folder names
    frame_storage_folders = ['class1_frames', 'class2_frames']

    # Create directories to store frames for each class
    #for frame_storage_folder in frame_storage_folders:
    #    os.makedirs(frame_storage_folder, exist_ok=True)

    # Extract frames from videos in each class
   # for class_folder, frame_storage_folder in zip(class_folders, frame_storage_folders):
   #     class_folder_path = os.path.join(folder, class_folder)
   #     for video_filename in os.listdir(class_folder_path):
        #    video_path = os.path.join(class_folder_path, video_filename)
         #   extract_frames(video_path, frame_storage_folder)

    # Prepare datasets for each class
    datasets = []
    for i, frame_storage_folder in enumerate(frame_storage_folders):
        class_data = prepare_dataset(frame_storage_folder, i)
        datasets.extend(class_data)

    # Combine and shuffle the dataset
    np.random.shuffle(datasets)

    # Split features and labels
    X = np.array([item[0] for item in datasets])
    y = np.array([item[1] for item in datasets])

    # Convert to PyTorch tensors
    features = torch.tensor(X, dtype=torch.float32)
    labels = torch.tensor(y, dtype=torch.float32).unsqueeze(1) #1
    dataset = TensorDataset(features, labels)

    # Split into training and testing datasets
    train_size = int(0.8 * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    # Initialize the model, loss function, and optimizer
    input_dim = 66  # Assuming 33 keypoints with x and y coordinates
    hidden_dim = 128
    output_dim = 2

    model = PoseLSTMClassifier(input_dim, hidden_dim, output_dim)
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Training loop
    num_epochs = 100
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        for batch_features, batch_labels in train_loader:
            optimizer.zero_grad()
            outputs = model(batch_features)
            loss = criterion(outputs, batch_labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

            # Calculate accuracy
            predicted = (outputs > 0.5).float()
            train_correct += (predicted == batch_labels).sum().item()
            train_total += batch_labels.size(0)

        train_loss /= len(train_loader)
        train_accuracy = train_correct / train_total

        # Evaluate on the test set
        model.eval()
        test_loss = 0.0
        test_correct = 0
        test_total = 0
        with torch.no_grad():
            for batch_features, batch_labels in test_loader:
                outputs = model(batch_features)
                loss = criterion(outputs, batch_labels)
                test_loss += loss.item()

                predicted = (outputs > 0.5).float()
                test_correct += (predicted == batch_labels).sum().item()
                test_total += batch_labels.size(0)

        test_loss /= len(test_loader)
        test_accuracy = test_correct / test_total

        print(f'Epoch {epoch+1}, Train Loss: {train_loss:.6f}, Train Accuracy: {train_accuracy:.2f}, Test Loss: {test_loss:.6f}, Test Accuracy: {test_accuracy:.2f}')

    # Save the trained model
    torch.save(model.state_dict(), 'pose_lstm_classifier.pth')

IndexError: Dimension out of range (expected to be in range of [-2, 1], but got 2)